In [ ]:
import pandas as pd
import requests
import json

### Read initial data ###

In [ ]:
path_to_csv = 'volodymyr_graphs/graph_test_1.csv'

data = pd.read_csv(path_to_csv) 
data = data.sort_values(by='modularity_class')
data = data.drop(columns=['timeset'])
data.head()

### Query summaries and titles###

In [ ]:
df = pd.DataFrame(columns=['id', 'title', 'extract'])

# Ids from the initial graph
ids = sorted(data.Id.tolist())

for idd in ids:
    # Change 'en' to 'fr' for french , to 'ar' for arabic 
    base_url = 'https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&redirects=1&pageids='
    url = base_url + str(idd)
    res = requests.get(url)
    response = json.loads(res.text)
    for key in response['query']['pages']:
        try:
            pageid = response['query']['pages'][key]['pageid']
            title = response['query']['pages'][key]['title']
            extract = response['query']['pages'][key]['extract']
            
            df = df.append({'id': pageid, 'title': title, 'extract': extract}, ignore_index=True)
        except KeyError as e:
            print(e, key)

Save it for later :

In [ ]:
path_to_extracts = path_to_csv.split('.')[0]+ '_extracts.' + path_to_csv.split('.')[1]
df.to_csv(path_to_extracts)

### Get the final dataset

In [ ]:
data_e = pd.read_csv(path_to_extracts) #dataset with scraped extracts (summary of page)from web 

df_final = data.join(data_e.set_index('id'), on='Id')
df_final = df_final.dropna()
df_final = df_final.drop(columns=['Label'])
df_final = df_final.drop(columns=['Unnamed: 0'])

df_final.head()